<a href="https://colab.research.google.com/github/SYEON9/natural_language_3th/blob/main/HW/%5BHW12%5DLogistic_Regression_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Linear regression은 주어진 데이터와 가장 잘 맞는 직선을 찾는 방법이다. 하지만 예측값이 연속적인 값을 갖지 않는다면 어떻게 해야할까??
여러 class를 분류해야하는 문제를 해결하려면 logistic regression을 사용해야 한다. 

In [ ]:
#Logistic regression
import sympy
import numpy

from matplotlib import pyplot
%matplotlib inline

In [ ]:
z = sympy.Symbol('z', real=True)

logistic = 1/(1+sympy.exp(-z))
logistic

In [ ]:
sympy.plotting.plot(logistic)

In [ ]:
#이제 데이터를 직접 만들어서 실습해보자.
#synthetic data(합성 데이터)
x_data = numpy.linspace(-5,5,100)
w = 2
b = 1
numpy.random.seed(0)
z_data = b + w*x_data + numpy.random.normal(size=len(x_data))
y_data = 1/(1+numpy.exp(-z_data))

pyplot.scatter(x_data, y_data, alpha=0.4)

In [ ]:
#이제 실제 class값을 정해주자.
#0.5보다 크면 1으로 작으면 0으로 class를 부여한다.
y_data = numpy.where(y_data >= 0.5, 1, 0)
pyplot.scatter(x_data, y_data, alpha=0.4);

In [ ]:
#최적의 가중치를 찾기 위해서는 cost function을 정의하고 loss를 최소화해야한다. 
#그런데 이 cost function은 linear regression과 달리 logistic regression은 더이상 convex하지 않다.
#이것은 sigmoid function 때문인데 실습으로 직접 확인해보자. 

badloss = (2 - 1/(1 + sympy.exp(-z)))**2 + \
          (-1 - 1/(1 + sympy.exp(-20*z)))**2 + \
          (5 - 1/(1 + sympy.exp(-5*z)))**2 
badloss


In [ ]:
sympy.plotting.plot(badloss, xlim = (-1,1));

In [ ]:
#gradient descent 방법으로 위 cost function의 최솟값을 구하게 되면 중간에 기울기가 0인 지점에서 멈춘다.
#즉, cost가 가장 작은 값에 도달하지 못하고 local minumum에 도달하게 된다. 
#그러므로 가중치에 대한 편미분으로 값을 각각 구해보도록 하겠다.

Iprime = logistic.diff(z)
Iprime

In [ ]:
a, y = sympy.symbols('a y', real = True)

In [ ]:
dLda = (y - a)/a/(1-a)                  ##?????? 왜 (y-a)/(a*(1-a))가 아니지?
dLda

In [ ]:
L = sympy.integrate(dLda, a)
L

In [ ]:
L = -y*sympy.log(a) + (y-1)*sympy.log(1-a)
L

In [ ]:
#y=1인 경우의 cost값
sympy.plotting.plot(-sympy.log(a), xlim = (0,1));

In [ ]:
#y=0인 경우의 cost값
sympy.plotting.plot(-sympy.log(1-a), xlim = (0,1));

In [ ]:
#이제 전체적인 과정을 따라가보면서 알아보자.
logistic

In [ ]:
w, b, x, y = sympy.symbols('w b x y')
logistic = 1/(1 + sympy.exp(-w*x -b))

Loss = -y*sympy.log(logistic) - (1-y)*sympy.log(1-logistic)
Loss

In [ ]:
#가중치 구하기(==기울기 구하기 == cost function 미분하기)
from autograd import numpy
from autograd import grad

In [ ]:
#function define

def logistic(z):
    '''The logistic function'''
    return 1 / (1 + numpy.exp(-z))

def logistic_model(params, x):
    '''A prediction model based on the logistic function composed with wx+b
    Arguments:
      params: array(w,b) of model paramsters
      x: array of x data
    '''
    w = params[0]
    b = params[1]
    z = w * x + b
    y = logistic(z)
    return y

def log_loss(params, model, x, y):
    '''The logistic loss function 
    Arguuments:
      params: array(w, b) of model parameters
      model: the Python function for the logistic model
      x, y: arrays of input data to the model
    '''
    y_pred = model(params, x)
    return -numpy.mean(y * numpy.log(y_pred) + (1-y) * numpy.log(1-y_pred))

In [ ]:
#logistic loss의 gradient를 계산한 값을 얻자!
gradient = grad(log_loss)                      

In [ ]:
type(gradient)

In [ ]:
#grad()는 변수 개수만큼 output을 만든다. 
#우리는 w,b라는 변수 2개가 있으므로 기울기값을 구해보자
numpy.random.seed(10)
params = numpy.random.rand(2)      #params를 임의 설정
print(params)

In [ ]:
#기울기값을 구하자
gradient(params, logistic_model, x_data, y_data)

In [ ]:
#gradient descent를 진행할때 반복수를 전부 채우지 않아도 기울기값이 0에 가까워지면 더 이상 반복하지 않도록 설정하자.
max_iter = 5000
i = 0
descent = numpy.ones(len(params))

while numpy.linalg.norm(descent) > 0.001 and i < max_iter:

    descent = gradient(params, logistic_model, x_data, y_data)
    params = params - descent * 0.01
    i += 1

print('Optimized value of w is {} vs. true value:2'.format(params[0]))
print('Optimized value of b is {} vs. true value 1:'.format(params[1]))
print('Exited after {} iterations'.format(i))

pyplot.scatter(x_data, y_data, alpha = 0.4)
pyplot.plot(x_data, logistic_model(params, x_data), '-r');

In [ ]:
#class로 나눠보자.
def decision_boundary(y):
    return 1 if y>=0.5 else 0

In [ ]:
decision_boundary = numpy.vectorize(decision_boundary)

In [ ]:
def classify(predictions):
    '''
    Argument:
    predictions, an array of values between 0 and 1
    
    Returns: 
    classified, an array of 0 and 1 values'''

    return decision_boundary(predictions).flatten()

pyplot.scatter(x_data, y_data, alpha=0.4,
               label='true value')
pyplot.scatter(x_data, classify(logistic_model(params, x_data)), alpha=0.4, 
               label='prediciton')

pyplot.legend();